# Data Cleaning + Collection

## API Calls via NASA POWER API + NHC Hurricane Track Paths
##### This data is collected for 15 data points located across the Hurricane Major Development Region (MDR) in the Atlantic Ocean / Caribbean Sea (10-20°N , 20-80°N), with buffer added to 25°N and 85°W for expanded coverage. 18 Features are collected for each data point on each day. Data collection initially includes all data from 1984-2025, however, is narrowed in coverage from 2001-2024 as all features are serviced starting in 2001 includng many new additonal metrics.

###### All of the following cells have not been run in this file upload as API calls already made and DataFrame was saves locally as a .csv , this file highlights the collection, cleaning, and merging process for replication.

### 1. Single Data Point Test

In [2]:
#imports
import requests
import numpy as np
import pandas as pd

In [ ]:
#data range paramters
start_d=19840101
end_d=19841231

#location coordinates
lat,lon=10,-85

#features for each data point
my_params="T2M,T10M,T2M_RANGE,T10M_RANGE,RH2M,WS2M,WS50M,T2MWET,WD2M,WD50M,CLOUD_AMT,WS2M_RANGE,WS50M_RANGE,CLOUD_OD,PS,SLP,PW,PRECTOT"

#pull from API, use AG (Agricultural) community for data pulls
url= f"https://power.larc.nasa.gov/api/temporal/daily/point?parameters={my_params}&community=AG&longitude={lon}&latitude={lat}&start={start_d}&end={end_d}&format=JSON"

r=requests.get(url)
data=r.json()

##### To View Data Features Long Names:

In [ ]:
data["parameters"]

##### If Empty List API Call Successful:

In [ ]:
data["messages"]

##### To Make Simple DataFrame of Results:

In [ ]:
my_data=data['properties']
my_data=my_data['parameter']
df=pd.DataFrame(my_data)
df.index.name="date"
df["lat"]=lat
df["lon"]=lon
df["location"]=1
df

### 2. Full NASA POWER API Call
##### This call takes approximately 20 minutes to complete. API has limit of 30 requests per hour, this cell requires 15 total (1 for each location).

In [ ]:
#parameters being used
full_params="T2M,T10M,T2M_RANGE,T10M_RANGE,RH2M,WS2M,WS50M,T2MWET,WD2M,WD50M,CLOUD_AMT,WS2M_RANGE,WS50M_RANGE,CLOUD_OD,PS,SLP,PW,PRECTOT"

#full data from start of 1984 to end of 2024
start_d=19840101
end_d=20241231

#Selecting 15 points over Carribean Sea & Atlantic Ocean, using NDAA's Main Development Region (MDR), 10°N to 20°N latitude and 20°W to 80°W longitude, used up to 25°N to include The Bahamas as point of interest and 5° buffer for longitude 
my_points=[]
for i in range(3):
    for j in range(5):
        my_points.append((10+7.5*i,(-85+((85-25)/4)*j),5*i+j+1))

#header indicator only to be written on first loc call
my_h=True

#loop for all 15 points
for lat,lon,loc_id in my_points:
    url= f"https://power.larc.nasa.gov/api/temporal/daily/point?parameters={full_params}&community=AG&longitude={lon}&latitude={lat}&start={start_d}&end={end_d}&format=JSON"
    r=requests.get(url)
    data=r.json()
    #error handling, print statement indicates issue with API call
    if "properties" not in data:
        print(loc_id)
        continue
    my_data=data['properties']
    my_data=my_data['parameter']
    df=pd.DataFrame(my_data)
    df.index.name="date"
    df.reset_index(inplace=True)
    df["lat"]=lat
    df["lon"]=lon
    df["location"]=loc_id
    my_h=False
    
df=df.sort_values("date")

##### Data contained in file locally, loaded to display outputs, but not necessary if entire file run at once.

In [3]:
# df=pd.read_csv("weather_data_sorted.csv")

##### In cleaning ran these cells to discover feature coverage starting 2001 was best, API returns use -999 in place of missing data. 

In [4]:
#copy df to avoid pandas warning about copy of a slice
df=df.copy()
df.replace(-999,np.nan,inplace=True)
df=df.dropna()

##### Hurricane Season covers June 1st - Novemeber 31st, only looking at these relevent dates (65,880 rows of data)

In [5]:
df["month"]=df["date"].astype(str).str[4:6].astype(int)
df=df[df["month"]>=6]
df=df[df["month"]<=11]
df=df.drop(columns=["month"])
df

,date,T2M,T10M_RANGE,WD50M,WS2M_RANGE,WD2M,T2MWET,T2M_RANGE,WS2M,CLOUD_AMT,...,PS,WS50M_RANGE,T10M,WS50M,PW,RH2M,PRECTOTCORR,lat,lon,location
95415,20010601,0.97,80.80,0.70,101.66,25.39,5.64,2.87,22.50,101.66,...,2.03,70.21,0.43,0.39,80.80,27.97,0.00,17.5,-40.0,9
95416,20010601,27.34,0.53,73.10,1.16,72.90,25.07,0.46,6.50,52.01,...,101.37,1.64,27.29,8.34,3.76,76.19,0.45,10.0,-55.0,3
95417,20010601,3.88,3.19,1.33,2.25,101.81,20.42,21.30,22.71,3.31,...,2.42,0.78,13.86,74.51,101.81,21.60,0.00,25.0,-25.0,15
95418,20010601,24.34,102.90,101.73,0.71,3.14,26.14,0.66,26.21,79.86,...,77.77,4.33,1.82,3.65,2.61,101.73,0.66,25.0,-70.0,12
95419,20010601,7.46,1.40,0.33,3.39,101.39,24.04,59.70,25.76,6.02,...,0.96,3.07,58.65,79.87,101.39,59.30,0.28,10.0,-40.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224170,20241130,84.40,28.08,2.58,43.92,10.02,101.63,1.68,28.02,0.30,...,101.63,7.62,26.00,78.11,84.10,1.14,1.51,17.5,-55.0,8
224171,20241130,17.70,79.38,18.20,1.56,28.06,94.99,5.77,101.32,8.17,...,0.49,101.32,0.88,10.91,10.90,26.15,55.12,17.5,-85.0,6
224172,20241130,5.06,3.03,9.95,4.31,100.89,21.24,118.90,25.96,2.24,...,3.22,6.84,26.30,56.40,90.87,118.40,0.00,10.0,-70.0,2
224173,20241130,97.90,82.69,98.50,0.86,28.74,45.44,5.45,101.33,4.88,...,1.10,101.33,0.60,0.95,6.02,27.18,3.72,17.5,-70.0,7


### Input Features Map (Acronym - Full Name)

- **T2M** - 2 Meter Air Temperature  
- **T10M** - 10 Meter Air Temperature  
- **T2M_RANGE** - Daily Range of 2 Meter Air Temperature  
- **T10M_RANGE** - Daily Range of 10 Meter Air Temperature  
- **RH2M** - 2 Meter Relative Humidity  
- **WS2M** - 2 Meter Wind Speed  
- **WS50M** - 50 Meter Wind Speed  
- **T2MWET** - 2 Meter Wet Bulb Temperature  
- **WD2M** - 2 Meter Wind Direction  
- **WD50M** - 50 Meter Wind Direction  
- **CLOUD_AMT** - Total Cloud Amount  
- **WS2M_RANGE** - Daily Range of 2 Meter Wind Speed  
- **WS50M_RANGE** - Daily Range of 50 Meter Wind Speed  
- **CLOUD_OD** - Cloud Optical Depth  
- **PS** - Surface Pressure  
- **SLP** - Sea Level Pressure  
- **PW** - Precipitable Water  
- **PRECTOT** - Total Precipitation  

### 3. National Hurricane Center (NHC) Hurricane Track Paths
##### This data can be found via this url: https://www.nhc.noaa.gov/data/hurdat/hurdat2-1851-2024-040425.txt
##### Data is contained in .txt format, it was downloaded in csv format and cleaned below. Each hurricane has its dates of occurence and longitude and latitude coordinates at time intervals ranging between 2-4x per day. This track data was used to add binary indicators to my selected locations indicating whether or not a hurricane was within various distance thresholds of that point on a specific day. I used 100, 250, 500, and 750 mile thresholds. 
##### Storms include Hurricanes, as well as lower level storms such as Tropical Storms and Tropical Depressions, I elected to only include Hurricanes and Tropical Storms (the 2 strongest categories of storm). Tropcial Storms were included to offset the (still large) class imbalance between the positive (storm occuring) and negative (no storm occuring) class. The process is seen below:

##### Reading file and creating DataFrame, logic includes removal of rows containing header for each storm name.

In [ ]:
with open("hurricane_paths.csv","r") as f:
    my_lines=f.readlines()
tracks=[line for line in my_lines if line[:8].isdigit()] #if name of hurricane row is not a date so not digit, removes all
tracks=[line.split(",") for line in tracks]
hf=pd.DataFrame(tracks)
#only relevant cols, date, storm severity, lat, lon, keep these and add names
hf=hf.iloc[:,[0,3,4,5]]
hf.columns=["date","storm_severity","lat","lon"]

##### Since we are only using dates after 2001, filtering for these.

In [ ]:
hf=hf[hf["date"].astype(int)>20010601]

##### To view breakdown of storm severity:

In [ ]:
hf["storm_severity"].value_counts()

##### Only used HU (Hurricanes) and TS (Tropical Storms)

In [ ]:
hf["storm_severity"]=hf["storm_severity"].astype(str).str.strip()#fixing issue with text type
hf["lat"]=hf["lat"].astype(str).str.strip()
hf["lon"]=hf["lon"].astype(str).str.strip()
hf=hf[hf["storm_severity"].isin(["TS","HU"])]
#removing indicator for cardinal direction
hf["lat"] = hf["lat"].str[:-1].astype(float)
hf["lon"] = hf["lon"].str[:-1].astype(float)
hf

### 4. Merging Datasets
##### To join the data I used the haversine distance and the aforementioned distance thresholds.

##### First joining data as DataFrames to match efficiently as opposed to looping process:

In [ ]:
# pip install haversine #if needed
from haversine import haversine, Unit

#for loop too slow so will merge the two dfs on an outer join on the date
#format is same but datatypes differed so making both strings
df["date"]=df["date"].astype(str)
hf["date"]=hf["date"].astype(str)

merged=df.merge(hf,on="date",how="left",suffixes=("_loc","_storm"))

##### Sometimes there are multiple storms on the same day, as well as multiple locations of the same storm per day. Removing so that DataFrame only has one entry per location per day via taking the closest possible storm distance that day as the distance value. Days with no storm have no data on the merge, their distance is set as a threshold of 1,000,000 miles so as to never trigger indicators.

In [ ]:
merged["distance_miles"]=merged.apply(lambda x:haversine((x.lat_loc,x.lon_loc),(x.lat_storm,-x.lon_storm),unit=Unit.MILES),axis=1)

#any days with no storms will have nans from join
#need to assign high value for any locs with nan for dist since no hurricane on those days
merged["distance_miles"]=merged["distance_miles"].fillna(1000000)
mask=merged.groupby(["date","location"])["distance_miles"].idxmin()
agg_df=merged.loc[mask].reset_index(drop=True)
agg_df.to_csv("hurricane_track_distance_points.csv",index=False)

#saving progress and reloading
# agg_df.to_csv("hurricane_track_distance_points.csv",index=False)

In [16]:
df=pd.read_csv("hurricane_track_distance_points.csv")
df

,date,T2M,T10M_RANGE,WD50M,WS2M_RANGE,WD2M,T2MWET,T2M_RANGE,WS2M,CLOUD_AMT,...,PW,RH2M,PRECTOTCORR,lat_loc,lon_loc,location,storm_severity,lat_storm,lon_storm,distance_miles
0,20010601,25.48,2.62,229.30,1.84,229.90,24.10,3.37,1.94,99.98,...,4.93,85.06,7.50,10.0,-85.0,1,NaN,NaN,NaN,1000000.0
1,20010601,3.78,2.34,5.69,4.45,101.05,20.36,111.90,21.87,1.47,...,90.99,106.50,3.27,10.0,-70.0,2,NaN,NaN,NaN,1000000.0
2,20010601,27.34,0.53,73.10,1.16,72.90,25.07,0.46,6.50,52.01,...,3.76,76.19,0.45,10.0,-55.0,3,NaN,NaN,NaN,1000000.0
3,20010601,7.46,1.40,0.33,3.39,101.39,24.04,59.70,25.76,6.02,...,101.39,59.30,0.28,10.0,-40.0,4,NaN,NaN,NaN,1000000.0
4,20010601,84.81,66.84,20.40,101.38,3.61,20.30,4.23,24.60,0.39,...,0.37,2.48,0.00,10.0,-25.0,5,NaN,NaN,NaN,1000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65875,20241130,75.86,101.89,26.10,76.82,4.55,23.42,25.80,3.33,23.81,...,8.72,21.65,0.43,25.0,-85.0,11,NaN,NaN,NaN,1000000.0
65876,20241130,25.18,108.20,101.51,0.83,1.70,26.31,0.78,26.32,87.20,...,3.10,101.51,8.32,25.0,-70.0,12,NaN,NaN,NaN,1000000.0
65877,20241130,25.83,0.54,120.80,1.98,120.40,23.54,0.48,5.94,50.64,...,3.60,75.77,1.66,25.0,-55.0,13,NaN,NaN,NaN,1000000.0
65878,20241130,68.70,79.05,69.10,1.90,24.33,59.83,2.55,102.34,6.99,...,8.79,22.62,0.44,25.0,-40.0,14,NaN,NaN,NaN,1000000.0


### 5. Distance Thresholds + Indicator Window (3-5 Days Before Storm)
##### Adding binary indicator for distance thresholds, and shifting time window for days to inlcude indicator 3-5 time window in advance of storm. This provides predictive power to warn before storm occurence with a reasonable enough time frame for preparations and further monitoring. Days of actual hurricane occurence, the preceding day, and the day 2 days before the storm are removed from the dataset to avoid data leakage.

##### Within 100 Miles:

In [17]:
df["within_100"]=(df["distance_miles"]<=100).astype("int")
df["within_100"].value_counts()

within_100
0    65697
1      183
Name: count, dtype: int64

##### Within 250 Miles:

In [18]:
df["within_250"]=(df["distance_miles"]<=250).astype("int")
df["within_250"].value_counts()

within_250
0    65172
1      708
Name: count, dtype: int64

##### Within 500 Miles:

In [19]:
df["within_500"]=(df["distance_miles"]<=500).astype("int")
df["within_500"].value_counts()

within_500
0    63500
1     2380
Name: count, dtype: int64

##### Within 750 Miles:

In [20]:
df["within_750"]=(df["distance_miles"]<=750).astype("int")
df["within_750"].value_counts()

within_750
0    61213
1     4667
Name: count, dtype: int64

#### Value Count Breakdowns:
###### within_100: 0 = 65697 | 1 = 183
###### within_250: 0 = 65172 | 1 = 708
###### within_500: 0 = 63500 | 1 = 2380
###### within_750: 0 = 61213 | 1 = 4667

##### Shifting indicators to 3-5 days window, requires grouping by locations to ensure shifting is done to correct location and dates:

In [21]:
#utilizing pandas shift feature, must sort and group by locations to ensure each applies correctly as currently sorted by date so standard shift would not work as intended
df=df.sort_values(["location", "date"])
#copying below for each dist

#masks for indexes 3 4 and 5 days, so shift separtely without overriding prior mask
df["in3_100"]=(df.groupby("location")["within_100"].shift(-3))
df["in4_100"]=(df.groupby("location")["within_100"].shift(-4))
df["in5_100"]=(df.groupby("location")["within_100"].shift(-5))
#apply 1s in all correct indexes
df["in3-5_100_miles"]=((df["in3_100"] == 1) | (df["in4_100"] == 1) | (df["in5_100"] == 1)).astype(int)
#remove the mask cols as no longer needed
df.drop(columns=["in3_100", "in4_100", "in5_100"], inplace=True)

#masks for indexes 3 4 and 5 days, so shift separtely without overriding prior mask
df["in3_250"]=(df.groupby("location")["within_250"].shift(-3))
df["in4_250"]=(df.groupby("location")["within_250"].shift(-4))
df["in5_250"]=(df.groupby("location")["within_250"].shift(-5))
#apply 1s in all correct indexes
df["in3-5_250_miles"]=((df["in3_250"] == 1) | (df["in4_250"] == 1) | (df["in5_250"] == 1)).astype(int)
#remove the mask cols as no longer needed
df.drop(columns=["in3_250", "in4_250", "in5_250"], inplace=True)

#masks for indexes 3 4 and 5 days, so shift separtely without overriding prior mask
df["in3_500"]=(df.groupby("location")["within_500"].shift(-3))
df["in4_500"]=(df.groupby("location")["within_500"].shift(-4))
df["in5_500"]=(df.groupby("location")["within_500"].shift(-5))
#apply 1s in all correct indexes
df["in3-5_500_miles"]=((df["in3_500"] == 1) | (df["in4_500"] == 1) | (df["in5_500"] == 1)).astype(int)
#remove the mask cols as no longer needed
df.drop(columns=["in3_500", "in4_500", "in5_500"], inplace=True)

#masks for indexes 3 4 and 5 days, so shift separtely without overriding prior mask
df["in3_750"]=(df.groupby("location")["within_750"].shift(-3))
df["in4_750"]=(df.groupby("location")["within_750"].shift(-4))
df["in5_750"]=(df.groupby("location")["within_750"].shift(-5))
#apply 1s in all correct indexes
df["in3-5_750_miles"]=((df["in3_750"] == 1) | (df["in4_750"] == 1) | (df["in5_750"] == 1)).astype(int)
#remove the mask cols as no longer needed
df.drop(columns=["in3_750", "in4_750", "in5_750"], inplace=True)

##### Indicators for removing days of actual hurricane as well as the preceding 1st and 2nd day that are not in our window, and the 1st and 2nd day following storm, as conditions likely to be similar to preceding 2 days and weaken the model's predictive strength. Not removed yet until engineered features are added:

In [22]:
#same process as before but shifting remove indicator only 0,1,2,-1,-2

#utilizing pandas shift feature, must sort and group by locations to ensure each applies correctly as currently sorted by date so standard shift would not work as intended
df=df.sort_values(["location", "date"])
#copying below for each dist

df["in0_100"]=(df.groupby("location")["within_100"].shift(-0))
df["in1_100"]=(df.groupby("location")["within_100"].shift(-1))
df["in2_100"]=(df.groupby("location")["within_100"].shift(-2))
df["af1_100"]=(df.groupby("location")["within_100"].shift(1))
df["af2_100"]=(df.groupby("location")["within_100"].shift(2))
#apply 1s in all correct indexes
df["remove_100"]=((df["in0_100"] == 1) | (df["in1_100"] == 1) | (df["in2_100"] == 1) | (df["af1_100"]==1) | (df["af2_100"]==1)).astype(int)
#remove the mask cols as no longer needed
df.drop(columns=["in0_100", "in1_100", "in2_100","af1_100","af2_100"], inplace=True)

df["in0_250"]=(df.groupby("location")["within_250"].shift(-0))
df["in1_250"]=(df.groupby("location")["within_250"].shift(-1))
df["in2_250"]=(df.groupby("location")["within_250"].shift(-2))
df["af1_250"]=(df.groupby("location")["within_250"].shift(1))
df["af2_250"]=(df.groupby("location")["within_250"].shift(2))
#apply 1s in all correct indexes
df["remove_250"]=((df["in0_250"] == 1) | (df["in1_250"] == 1) | (df["in2_250"] == 1) | (df["af1_250"]==1) | (df["af2_250"]==1)).astype(int)
#remove the mask cols as no longer needed
df.drop(columns=["in0_250", "in1_250", "in2_250","af1_250","af2_250"], inplace=True)

df["in0_500"]=(df.groupby("location")["within_500"].shift(-0))
df["in1_500"]=(df.groupby("location")["within_500"].shift(-1))
df["in2_500"]=(df.groupby("location")["within_500"].shift(-2))
df["af1_500"]=(df.groupby("location")["within_500"].shift(1))
df["af2_500"]=(df.groupby("location")["within_500"].shift(2))
#apply 1s in all correct indexes
df["remove_500"]=((df["in0_500"] == 1) | (df["in1_500"] == 1) | (df["in2_500"] == 1) | (df["af1_500"]==1) | (df["af2_500"]==1)).astype(int)
#remove the mask cols as no longer needed
df.drop(columns=["in0_500", "in1_500", "in2_500","af1_500","af2_500"], inplace=True)

df["in0_750"]=(df.groupby("location")["within_750"].shift(-0))
df["in1_750"]=(df.groupby("location")["within_750"].shift(-1))
df["in2_750"]=(df.groupby("location")["within_750"].shift(-2))
df["af1_750"]=(df.groupby("location")["within_750"].shift(1))
df["af2_750"]=(df.groupby("location")["within_750"].shift(2))
#apply 1s in all correct indexes
df["remove_750"]=((df["in0_750"] == 1) | (df["in1_750"] == 1) | (df["in2_750"] == 1) | (df["af1_750"]==1) | (df["af2_750"]==1)).astype(int)
#remove the mask cols as no longer needed
df.drop(columns=["in0_750", "in1_750", "in2_750","af1_750","af2_750"], inplace=True)

##### Final DataFrame:

In [28]:
# df.to_csv("hurricane_init.csv",index=False)
df

,date,T2M,T10M_RANGE,WD50M,WS2M_RANGE,WD2M,T2MWET,T2M_RANGE,WS2M,CLOUD_AMT,...,within_500,within_750,in3-5_100_miles,in3-5_250_miles,in3-5_500_miles,in3-5_750_miles,remove_100,remove_250,remove_500,remove_750
0,20010601,25.48,2.62,229.30,1.84,229.90,24.10,3.37,1.94,99.98,...,0,0,0,0,0,0,0,0,0,0
15,20010602,25.76,2.91,223.50,1.92,219.90,24.27,3.57,2.10,93.27,...,0,0,0,0,0,0,0,0,0,0
30,20010603,25.53,3.19,270.50,2.30,267.40,23.97,3.87,1.73,85.00,...,0,0,0,0,0,0,0,0,0,0
45,20010604,24.99,1.89,253.80,1.17,250.80,24.03,2.35,2.39,83.00,...,0,0,0,0,0,0,0,0,0,0
60,20010605,25.86,3.67,256.80,2.11,249.70,24.42,4.14,1.58,86.38,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65819,20241126,3.01,2.33,0.40,2.71,101.87,20.71,74.10,22.62,2.69,...,0,0,0,0,0,0,0,0,0,0
65834,20241127,1.41,2.66,0.61,2.92,101.71,20.48,146.10,22.60,1.29,...,0,0,0,0,0,0,0,0,0,0
65849,20241128,1.27,2.26,0.85,2.78,101.75,20.59,103.20,22.83,1.17,...,0,0,0,0,0,0,0,0,0,0
65864,20241129,3.02,3.45,0.79,2.88,101.90,21.11,61.90,23.13,2.67,...,0,0,0,0,0,0,0,0,0,0
